In [ ]:
!pip install pythainlp
!pip install emoji

In [ ]:
import numpy as np
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt

In [ ]:
import tensorflow as tf
from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.preprocessing.sequence import pad_sequences
from tensorflow.keras import layers
from pythainlp.ulmfit import *

In [ ]:
import pandas as pd
import re
# from nltk.tokenize import word_tokenize
from pythainlp.tokenize import word_tokenize
from tensorflow.keras.preprocessing.text import Tokenizer

from keras.preprocessing.sequence import pad_sequences
from sklearn.preprocessing import OneHotEncoder
from sklearn.model_selection import train_test_split

import numpy as np

from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, GRU, LSTM, Bidirectional, Embedding, Dropout, BatchNormalization
from tensorflow.keras.models import load_model
from tensorflow.keras.callbacks import ModelCheckpoint

from tensorflow.keras.optimizers import Adam

import seaborn as sn
import matplotlib.pyplot as plt

import pickle as p
import plotly
import plotly.graph_objs as go

from sklearn.metrics import confusion_matrix

from sklearn.metrics import classification_report

In [ ]:
with open("/content/train.txt") as f:
    texts = [line.strip() for line in f.readlines()]

with open("/content/train_label.txt") as f:
    categories = [line.strip() for line in f.readlines()]

all_df = pd.DataFrame({"texts":texts, "category":categories})
all_df.to_csv('train.csv',index=False)
all_df.shape

(24063, 2)

In [ ]:
import pandas as pd

df = pd.read_csv('/content/train.csv', encoding='utf-8')
df = df[df['category'] != 'q']

# Desired number of samples for each category
desired_neu_count = 7000  # Adjust this as needed

# Separate the "neu" category
neu_df = df[df['category'] == 'neu']

# Randomly sample from the "neu" category
sampled_neu_df = neu_df.sample(n=desired_neu_count, random_state=42)

# Keep all rows from the other categories
other_df = df[df['category'] != 'neu']

# Combine the downsampled "neu" with the rest of the data
balanced_df = pd.concat([sampled_neu_df, other_df])

# Shuffle the rows randomly
balanced_df = balanced_df.sample(frac=1, random_state=42).reset_index(drop=True)

# Check the new counts
print(balanced_df['category'].value_counts())


category
neu    7000
neg    6140
pos    4300
Name: count, dtype: int64


In [ ]:
balanced_df.head(2)

,texts,category
0,Toyota Corona AT190 ท้ายโด่งครับ,neu
1,เดวน้องออมกลับบ้านก่อนนะนุ้งยิ้ม,pos


In [ ]:
print(balanced_df.shape)
print(balanced_df.isnull().sum())
print(balanced_df.duplicated().sum())
print(balanced_df.info)

(17440, 2)
texts       0
category    0
dtype: int64
22
<bound method DataFrame.info of                                                    texts category
0                       Toyota Corona AT190 ท้ายโด่งครับ      neu
1                       เดวน้องออมกลับบ้านก่อนนะนุ้งยิ้ม      pos
2                                               ป๋าเตยยย      neu
3                      วีออสกับรีโว่ก็ออกทะเลทั้งคู่คราบ      neu
4      ทีแรกจะซื้อมาสด้าสองและซีเอ๊ก3 พร้อมกันสองคันเ...      neg
...                                                  ...      ...
17435  Graduate - School of Performing Arts Seoul (SO...      pos
17436                         จะรีบเสือกอย่างไว รอแปรบบบ      neg
17437  ธนาธร ไปโคราชเมื่อวาน (14 กุมภา 62) ส่วนสุเทพไ...      neu
17438     ขี้มอไซไปไง ที่โฮมโปรมีจะไปปะละ ใจๆ5555หิวด้วย      neu
17439  นิสสันแกร่งจริง ทนจริง ถ้าเป็นกระบะที่บ้านวางใ...      pos

[17440 rows x 2 columns]>


In [ ]:
balanced_df['category'].value_counts()

,count
category,
neu,7000
neg,6140
pos,4300


In [ ]:
balanced_df.dropna(inplace = True)

In [ ]:
balanced_df.head()

,texts,category
0,Toyota Corona AT190 ท้ายโด่งครับ,neu
1,เดวน้องออมกลับบ้านก่อนนะนุ้งยิ้ม,pos
2,ป๋าเตยยย,neu
3,วีออสกับรีโว่ก็ออกทะเลทั้งคู่คราบ,neu
4,ทีแรกจะซื้อมาสด้าสองและซีเอ๊ก3 พร้อมกันสองคันเ...,neg


In [ ]:
# for i in range(30):
#     print(balanced_df['texts'].iloc[i]) # Use iloc to access by position

In [ ]:
# import re
# def clean(text):
#     text = re.sub(r'[^ก-๙\s]', '', text)
#     # Remove extra whitespaces
#     text = re.sub(r'\s+', ' ', text).strip()
#     return text

In [ ]:
# # preprocess

# from pythainlp.tokenize import word_tokenize

# # Apply word_tokenize from PyThaiNLP to your text column
# balanced_df['processed_texts'] = balanced_df['texts'].apply(
#     lambda text: word_tokenize(text, engine="newmm", keep_whitespace=False)
# )

# # View the processed text column
# balanced_df['processed_texts']

# def create_tokenizer(words, filters = ''):
#     token = Tokenizer(filters=filters)
#     token.fit_on_texts(words)
#     return token

# cleaned_words = balanced_df['processed_texts'].tolist()
# train_word_tokenizer = create_tokenizer(cleaned_words)

# # Step 3: Extract vocabulary size and word index
# vocab_size = len(train_word_tokenizer.word_index) + 1
# train_word_tokenizer.word_index

# # Print vocabulary size and word index
# print("Vocabulary size:", vocab_size)
# print("Word Index:", train_word_tokenizer.word_index)

# def max_length(words):
#     return(len(max(words, key = len)))

# max_length = max_length(balanced_df['processed_texts'])
# max_length

# def encoding_doc(token, words):
#     return(token.texts_to_sequences(words))

# encoded_doc = encoding_doc(train_word_tokenizer, cleaned_words)

# print(cleaned_words[0])
# print(encoded_doc[0])

# def padding_doc(encoded_doc, max_length):
#     return(pad_sequences(encoded_doc, maxlen = max_length, padding = "post"))

# padded_doc = padding_doc(encoded_doc, max_length)
# print("Shape of padded docs = ",padded_doc.shape)

# print(cleaned_words[3])
# print(encoded_doc[3])
# print(padded_doc[3])

In [ ]:
from pythainlp.tokenize import word_tokenize

# Apply word_tokenize from PyThaiNLP to your text column
balanced_df['processed_texts'] = balanced_df['texts'].apply(
    lambda text: word_tokenize(text, engine="newmm", keep_whitespace=False)
)

# View the processed text column
balanced_df['processed_texts']


,processed_texts
0,"[Toyota, Corona, AT, 190, ท้าย, โด่ง, ครับ]"
1,"[เด, ว, น้อง, ออม, กลับบ้าน, ก่อน, นะ, นุ้ง, ย..."
2,"[ป๋า, เตย, ยย]"
3,"[วีออส, กับ, รี, โว่, ก็, ออก, ทะเล, ทั้งคู่, ..."
4,"[ทีแรก, จะ, ซื้อ, มาสด้า, สอง, และ, ซี, เอ๊, ก..."
...,...
17435,"[Graduate, -, School, of, Performing, Arts, Se..."
17436,"[จะ, รีบ, เสือก, อย่าง, ไว, รอ, แป, รบ, บบ]"
17437,"[ธนา, ธร, ไป, โคราช, เมื่อวาน, (14, กุมภา, 62,..."
17438,"[ขี้, มอ, ไซ, ไป, ไง, ที่, โฮมโปร, มี, จะ, ไป,..."


In [ ]:
def create_tokenizer(words, filters = ''):
    token = Tokenizer(filters=filters)
    token.fit_on_texts(words)
    return token

In [ ]:
cleaned_words = balanced_df['processed_texts'].tolist()
train_word_tokenizer = create_tokenizer(cleaned_words)

# Step 3: Extract vocabulary size and word index
vocab_size = len(train_word_tokenizer.word_index) + 1
train_word_tokenizer.word_index

# Print vocabulary size and word index
print("Vocabulary size:", vocab_size)
print("Word Index:", train_word_tokenizer.word_index)

Vocabulary size: 23801
Word Index: {'ไม่': 1, 'ที่': 2, 'ไป': 3, 'ได้': 4, 'มี': 5, 'มา': 6, 'เลย': 7, 'ก็': 8, 'จะ': 9, 'ๆ': 10, 'แล้ว': 11, 'กิน': 12, 'ให้': 13, 'แต่': 14, 'ครับ': 15, 'ของ': 16, 'ใน': 17, 'ว่า': 18, 'เป็น': 19, 'ค่ะ': 20, 'และ': 21, '#': 22, 'นี้': 23, 'นะ': 24, 'กับ': 25, 'มาก': 26, '-': 27, 'มัน': 28, 'ใช้': 29, '.': 30, 'ดี': 31, 'อยาก': 32, 'ต้อง': 33, 'เรา': 34, 'คน': 35, 'คือ': 36, 'ด้วย': 37, 'กัน': 38, 'รถ': 39, 'ยัง': 40, 'กว่า': 41, 'เบียร์': 42, 'อีก': 43, '2': 44, 'ถ้า': 45, 'ละ': 46, 'การ': 47, 'เพราะ': 48, 'เอา': 49, 'ผม': 50, 'ซื้อ': 51, 'นี่': 52, 'จาก': 53, 'อยู่': 54, 'แบบ': 55, 'ทำ': 56, '1': 57, ')': 58, '3': 59, 'ช้าง': 60, 'กู': 61, 'อะไร': 62, 'ชอบ': 63, 'วัน': 64, 'ดู': 65, 'ราคา': 66, 'บาท': 67, '(': 68, 'หรือ': 69, '"': 70, 'กก': 71, 'สาขา': 72, 'อย่าง': 73, 'ปี': 74, 'คุณ': 75, 'พี่': 76, 'ก้อน': 77, 'แม่': 78, 'แค่': 79, 'ถึง': 80, 'บอก': 81, ':': 82, 'หน้า': 83, 'ขอ': 84, 'หมด': 85, 'ก่อน': 86, 'สวย': 87, 'ตัว': 88, 'ง': 89, 'พอ': 90, 'เ

In [ ]:
def max_length(words):
    return(len(max(words, key = len)))

In [ ]:
max_length = max_length(balanced_df['processed_texts'])
max_length

440

In [ ]:
balanced_df.shape

(17440, 3)

In [ ]:
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Embedding, LSTM, Dense, Dropout, Bidirectional
from sklearn.model_selection import train_test_split
from sklearn.metrics import classification_report, confusion_matrix

In [ ]:
def encoding_doc(token, words):
    return(token.texts_to_sequences(words))

In [ ]:
encoded_doc = encoding_doc(train_word_tokenizer, cleaned_words)

print(cleaned_words[0])
print(encoded_doc[0])

['Toyota', 'Corona', 'AT', '190', 'ท้าย', 'โด่ง', 'ครับ']
[479, 12416, 1229, 3008, 1116, 9048, 15]


In [ ]:
def padding_doc(encoded_doc, max_length):
    return(pad_sequences(encoded_doc, maxlen = max_length, padding = "post"))

In [ ]:
padded_doc = padding_doc(encoded_doc, max_length)
print("Shape of padded docs = ",padded_doc.shape)

print(cleaned_words[3])
print(encoded_doc[3])
print(padded_doc[3])

Shape of padded docs =  (17440, 440)
['วีออส', 'กับ', 'รี', 'โว่', 'ก็', 'ออก', 'ทะเล', 'ทั้งคู่', 'คราบ']
[684, 25, 215, 569, 8, 149, 1531, 2062, 1172]
[ 684   25  215  569    8  149 1531 2062 1172    0    0    0    0    0
    0    0    0    0    0    0    0    0    0    0    0    0    0    0
    0    0    0    0    0    0    0    0    0    0    0    0    0    0
    0    0    0    0    0    0    0    0    0    0    0    0    0    0
    0    0    0    0    0    0    0    0    0    0    0    0    0    0
    0    0    0    0    0    0    0    0    0    0    0    0    0    0
    0    0    0    0    0    0    0    0    0    0    0    0    0    0
    0    0    0    0    0    0    0    0    0    0    0    0    0    0
    0    0    0    0    0    0    0    0    0    0    0    0    0    0
    0    0    0    0    0    0    0    0    0    0    0    0    0    0
    0    0    0    0    0    0    0    0    0    0    0    0    0    0
    0    0    0    0    0    0    0    0    0    0    0    0    0 

In [ ]:
# # Map categories to integers
# category_mapping = {'neu': 0, 'neg': 1, 'pos': 2}
# balanced_df['category_encoded'] = balanced_df['category'].map(category_mapping)

In [ ]:
unique_category = list(set(balanced_df['category']))
print("Unique Categories:", unique_category)

Unique Categories: ['pos', 'neg', 'neu']


In [ ]:
output_tokenizer = create_tokenizer(unique_category)

In [ ]:
encoded_output = encoding_doc(output_tokenizer, balanced_df['category'].tolist())
print(balanced_df['category'].tolist()[0:6])
print(encoded_output[0:6])

['neu', 'pos', 'neu', 'neu', 'neg', 'neg']
[[3], [1], [3], [3], [2], [2]]


In [ ]:
encoded_output = np.array(encoded_output).reshape(len(encoded_output), 1)
encoded_output.shape

(17440, 1)

In [ ]:
def one_hot(encode):
  oh = OneHotEncoder(sparse_output = False)
  return(oh.fit_transform(encode))

In [ ]:
output_one_hot = one_hot(encoded_output)
print(encoded_output[0])
print(output_one_hot[0])

[3]
[0. 0. 1.]


In [ ]:
train_X, val_X, train_Y, val_Y = train_test_split(padded_doc, output_one_hot, shuffle = True, test_size = 0.2, stratify=output_one_hot)

In [ ]:
# from tensorflow.keras.utils import to_categorical

# # One-hot encode the labels
# num_classes = len(category_mapping)  # Automatically detects the number of classes
# y_train = to_categorical(balanced_df['category_encoded'], num_classes=num_classes)
# x_test = to_categorical(balanced_df['category_encoded'], num_classes=num_classes)

# # Print shape and example
# print(y_train.shape)  # Should be (number_of_samples, 3)
# print(y_train[1])  # Example of one-hot vector, e.g., [1. 0. 0.]

In [ ]:
num_classes = len(unique_category)
num_classes

3

In [ ]:
# encoder = OneHotEncoder(sparse_output=False, handle_unknown='ignore')
# # The line below is changed
# labels = encoder.fit_transform(balanced_df['category'].values.reshape(-1, 1))
# # toarray() has been removed because it's already a NumPy array

# # Splitting data
# X_train, X_val, y_train, y_val = train_test_split(padded_doc, labels, test_size=0.3, random_state=42)
# # ... (rest of your code)
# model = Sequential([
#     Embedding(input_dim=17440, output_dim=128, input_length=440),
#     LSTM(128, return_sequences=False),
#     Dense(64, activation='relu'),
#     Dropout(0.5),
#     Dense(labels.shape[1], activation='softmax')
# ])

# # Compile the model
# model.compile(optimizer='adam', loss='categorical_crossentropy', metrics=['accuracy'])

# # Train the model
# history = model.fit(X_train, y_train, validation_data=(X_val, y_val), epochs=10, batch_size=32)

# # Evaluate the model
# print("Model Evaluation:")
# model.evaluate(X_val, y_val)

from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, GRU, LSTM, Bidirectional, Embedding, Dropout, BatchNormalization
from tensorflow.keras.models import load_model
from tensorflow.keras.layers import InputLayer

from tensorflow.keras.optimizers import Adam
adam = Adam(learning_rate=0.0001)

def create_model(vocab_size, max_length):
  model = Sequential()
  model.add(InputLayer(shape=(max_length,)))
  model.add(Embedding(vocab_size, 128,  trainable = True))
  model.add(Bidirectional(GRU(128, activation = "relu"))) # activation = "relu"
  model.add(Dense(128, activation = "relu"))
  model.add(Dropout(0.5))
  model.add(Dense(64, activation = "relu"))
  model.add(Dropout(0.5))
  model.add(BatchNormalization())
  model.add(Dense(num_classes, activation = "softmax"))

  return model

model = create_model(vocab_size, max_length)

In [ ]:
model.compile(loss = "categorical_crossentropy", optimizer = adam, metrics = ["accuracy"])
model.summary()

Model: "sequential"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━┓
┃ Layer (type)                         ┃ Output Shape                ┃         Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━┩
│ embedding (Embedding)                │ (None, 440, 128)            │       3,046,528 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ bidirectional (Bidirectional)        │ (None, 256)                 │         198,144 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ dense (Dense)                        │ (None, 128)                 │          32,896 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ dropout (Dropout)                    │ (None, 128)                 │               0 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ dense_1 (Dense)                      │ (None, 64)                  │           8,256 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ dropout_1 (Dropout)                  │ (None, 64)                  │               0 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ batch_normalization                  │ (None, 64)                  │             256 │
│ (BatchNormalization)                 │                             │                 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ dense_2 (Dense)                      │ (None, 3)                   │             195 │
└──────────────────────────────────────┴─────────────────────────────┴─────────────────┘

 Total params: 3,286,275 (12.54 MB)

 Trainable params: 3,286,147 (12.54 MB)

 Non-trainable params: 128 (512.00 B)

In [ ]:
filename = 'model.keras'
checkpoint = ModelCheckpoint(filename, monitor='val_loss', verbose=1, save_best_only=True, mode='min')

In [ ]:
EPOCHS = 10
BS = 32

In [ ]:
hist = model.fit(train_X, train_Y, epochs = EPOCHS, batch_size = BS, validation_data = (val_X, val_Y), callbacks = [checkpoint])

Epoch 1/10
436/436 ━━━━━━━━━━━━━━━━━━━━ 0s 169ms/step - accuracy: 0.5529 - loss: 0.9404
Epoch 1: val_loss improved from 1.05978 to 0.87049, saving model to model.keras
436/436 ━━━━━━━━━━━━━━━━━━━━ 78s 179ms/step - accuracy: 0.5529 - loss: 0.9403 - val_accuracy: 0.6018 - val_loss: 0.8705
Epoch 2/10
436/436 ━━━━━━━━━━━━━━━━━━━━ 0s 171ms/step - accuracy: 0.6432 - loss: 0.8340
Epoch 2: val_loss improved from 0.87049 to 0.80860, saving model to model.keras
436/436 ━━━━━━━━━━━━━━━━━━━━ 82s 179ms/step - accuracy: 0.6433 - loss: 0.8340 - val_accuracy: 0.6436 - val_loss: 0.8086
Epoch 3/10
436/436 ━━━━━━━━━━━━━━━━━━━━ 0s 169ms/step - accuracy: 0.7118 - loss: 0.6961
Epoch 3: val_loss improved from 0.80860 to 0.79312, saving model to model.keras
436/436 ━━━━━━━━━━━━━━━━━━━━ 81s 178ms/step - accuracy: 0.7118 - loss: 0.6961 - val_accuracy: 0.6583 - val_loss: 0.7931
Epoch 4/10
436/436 ━━━━━━━━━━━━━━━━━━━━ 0s 169ms/step - accuracy: 0.7674 - loss: 0.5917
Epoch 4: val_loss did not improve from 0.79312
4

In [ ]:
h1 = go.Scatter(y=hist.history['loss'], # Changed: hist to hist.history
                    mode="lines", line=dict(
                    width=2,
                    color='blue'),
                    name="loss"
                   )
h2 = go.Scatter(y=hist.history['val_loss'], # Changed: hist to hist.history
                    mode="lines", line=dict(
                    width=2,
                    color='red'),
                    name="val_loss"
                   )

data = [h1,h2]
layout1 = go.Layout(title='Loss',
                   xaxis=dict(title='epochs'),
                   yaxis=dict(title=''))
fig1 = go.Figure(data = data, layout=layout1)
fig1.show()

In [ ]:
h1 = go.Scatter(y=hist.history['accuracy'],
                    mode="lines", line=dict(
                    width=2,
                    color='blue'),
                    name="acc"
                   )
h2 = go.Scatter(y=hist.history['val_accuracy'],
                    mode="lines", line=dict(
                    width=2,
                    color='red'),
                    name="val_acc"
                   )

data = [h1,h2]
layout1 = go.Layout(title='Accuracy',
                   xaxis=dict(title='epochs'),
                   yaxis=dict(title=''))
fig1 = go.Figure(data = data, layout=layout1)
fig1.show()

In [ ]:
score = load_model(filename).evaluate(val_X, val_Y, verbose=0)
print('Validate loss:', score[0])
print('Validate accuracy:', score[1])

Validate loss: 0.7931178212165833
Validate accuracy: 0.6582568883895874


In [ ]:
# from tensorflow.keras.models import Sequential
# from tensorflow.keras.layers import Embedding, Bidirectional, LSTM, Dense, Dropout, BatchNormalization
# from tensorflow.keras.optimizers import Adam

# vocab_size = len(word_index) + 1
# embedding_dim = 300
# max_length = 407

# model = Sequential()
# model.add(Embedding(vocab_size, embedding_dim))
# model.add(Bidirectional(LSTM(64, return_sequences=True)))
# model.add(Dropout(0.5))
# model.add(Bidirectional(LSTM(64)))
# model.add(BatchNormalization())
# model.add(Dense(128, activation='relu'))
# model.add(Dropout(0.5))
# model.add(Dense(3, activation='softmax'))  # Assuming 3 output classes

# model.compile(
#     loss='categorical_crossentropy',
#     optimizer=Adam(learning_rate=0.001),
#     metrics=['accuracy']
# )

# model.fit(xtrian, ytrain, epochs=5, batch_size=32, validation_data=(xtest, ytest))


In [ ]:
# Save the model
model.save("my_model.h5")
print("Model saved as my_model.h5")

Model saved as my_model.h5


In [ ]:
from tensorflow.keras.models import load_model

# Load the trained model
model = load_model("model.keras")


In [ ]:
# from pythainlp.tokenize import word_tokenize
# from tensorflow.keras.preprocessing.sequence import pad_sequences

# # Replace 'word_index' with the actual word_index used in training
# word_index = train_word_tokenizer.word_index

# def preprocess_input(text, word_index, max_length):
#     # Tokenize input text
#     tokens = word_tokenize(text, engine='newmm', keep_whitespace=False)

#     # Convert tokens to sequences
#     sequence = [word_index.get(word, 0) for word in tokens]

#     # Pad sequence
#     padded_sequence = pad_sequences([sequence], maxlen=max_length, padding='post')
#     return padded_sequence


In [ ]:
from pythainlp.tokenize import word_tokenize
from tensorflow.keras.preprocessing.sequence import pad_sequences
import numpy as np
from tensorflow.keras.models import load_model

# Load the saved model
model = load_model("/content/model.keras")

# Assume train_word_tokenizer and max_length are loaded or redefined as follows:
word_index = train_word_tokenizer.word_index  # This should match the training tokenizer
max_length = max_length  # Use the same max_length from training

# Preprocessing function for new input
def preprocess_text(text, word_index, max_length):
    """
    Preprocess a single input text to match the model's expected input format.
    """
    # Step 1: Tokenize the input text
    tokens = word_tokenize(text, engine="newmm", keep_whitespace=False)

    # Step 2: Convert tokens to sequences using the training tokenizer's word_index
    sequence = [word_index.get(word, 0) for word in tokens]

    # Step 3: Pad the sequence to match the max_length used in training
    padded_sequence = pad_sequences([sequence], maxlen=max_length, padding="post")
    return padded_sequence

# Predict function
def predict_category(text, model, word_index, max_length, categories):
    """
    Predict the category of the input text using the trained model.
    """
    # Preprocess the input text
    processed_input = preprocess_text(text, word_index, max_length)

    # Predict using the model
    predictions = model.predict(processed_input)

    # Map the highest probability to the corresponding category
    predicted_category = categories[np.argmax(predictions)]

    return predicted_category, predictions

# Example Usage
categories = ["Positive", "Neutral", "Negative"]  # Adjust based on your label mapping
text_input = "ควยกระบือ"  # Replace with your input text

predicted_category, probabilities = predict_category(text_input, model, word_index, max_length, categories)
print(f"Input Text: {text_input}")
print(f"Predicted Category: {predicted_category}")
print(f"Prediction Probabilities: {probabilities}")


1/1 ━━━━━━━━━━━━━━━━━━━━ 1s 592ms/step
Input Text: ควยกระบือ
Predicted Category: Neutral
Prediction Probabilities: [[0.10913675 0.5181743  0.37268892]]


In [ ]:
# # Example text input
# text_input = "คุณนิสัยดี"  # Replace with your text
# processed_input = preprocess_input(text_input, word_index, max_length)

# # Predict using the loaded model
# predictions = model.predict(processed_input)

# # Map predictions to category labels
# categories = ["Positive", "Neutral", "Negative"]  # Adjust based on your training data
# predicted_category = categories[np.argmax(predictions)]

# print("Input Text:", text_input)
# print("Predicted Category:", predicted_category)


In [ ]:
# import numpy as np
# from tensorflow.keras.preprocessing.sequence import pad_sequences

# def predict_sentiment(text, model, word_index, max_length=407):
#     """
#     Predicts the sentiment of a given text.

#     Parameters:
#         text (str): The input text.
#         model (Sequential): The trained sentiment analysis model.
#         word_index (dict): The dictionary mapping words to indices.
#         max_length (int): The maximum length of input sequences (default: 407).

#     Returns:
#         str: The predicted sentiment label.
#     """
#     # Tokenize the input text
#     from pythainlp.tokenize import word_tokenize
#     tokens = word_tokenize(text, engine='newmm')

#     # Convert tokens to sequences of indices
#     sequence = [word_index.get(word, 0) for word in tokens]

#     # Pad the sequence to match the model's input length
#     padded_sequence = pad_sequences([sequence], maxlen=max_length, padding='post', truncating='post')

#     # Predict sentiment probabilities
#     predictions = model.predict(padded_sequence)

#     # Map probabilities to sentiment labels
#     sentiment_labels = ['Negative', 'Neutral', 'Positive']  # Adjust if different
#     sentiment = sentiment_labels[np.argmax(predictions)]

#     return sentiment, predictions

# # Example usage
# text = "บุหรี่"  # Replace with your input text
# sentiment = predict_sentiment(text, model, word_index)
# print("Predicted Sentiment:", sentiment)